### Step1: Data conversion
Read the BAG dataset, convert it to a geopaquet.

In [ ]:
from pathlib import Path
from datetime import datetime

# dask_geopandas, dask-expr, pyogrio
import dask_geopandas as dg

In [ ]:
# Get metadata
data_BAG = dg.read_file('../data/downloaded/BAG/bag-light-NL.gpkg', npartitions=50)
data_BAG

In [ ]:
# Create a path to save the data
path_BAG = Path('../data/dataset/BAG.parquet/')
path_BAG.mkdir(exist_ok=True, parents=True)

In [ ]:
# This does not work, complains about the data type of the geometry column
# data_BAG.to_parquet(path_BAG)
# data_BAG[['rdf_seealso', 'identificatie', 'bouwjaar', 'status', 'gebruiksdoel',
#        'oppervlakte_min', 'oppervlakte_max', 'aantal_verblijfsobjecten']].to_parquet(path_BAG) # This works because there is no geometry column

for i in range(data_BAG.npartitions):
    data_partition = data_BAG.get_partition(i).compute()
    data_partition.to_parquet(path_BAG / f'bag_{i}.parquet')

### Compair performace of gpkg and geopaquet

Test getting two columns from the BAG dataset, and then compute the first partition. Reading from parquet is almost 50 times faster than reading from geopackage.

In [ ]:
%%timeit
data_BAG_gpks = dg.read_file('../data/downloaded/BAG/bag-light-NL.gpkg', npartitions=50, columns=['bouwjaar', 'status'])
data_BAG_slice = data_BAG_gpks.get_partition(0).compute()

In [ ]:
%%timeit
data_BAG_parque = dg.read_parquet('../data/dataset/BAG.parquet/', columns=['bouwjaar', 'status'])
data_BAG_parque.get_partition(0).compute()